<a href="https://colab.research.google.com/github/blinovalex/lections_2019/blob/master/ChatBot_Day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import random
import nltk
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import train_test_split

In [11]:
with open('/content/BOT_CONFIG.json', 'r') as f:
  BOT_CONFIG = json.load(f)

FileNotFoundError: ignored

## День 1

In [9]:
# BOT_CONFIG = {
#     'intents': {
#         'hello': {
#             'examples': [
#                          'привет!',
#                          'хей',
#                          'Хай'
#             ],
#             'responses': [
#                           'прив',
#                           'добрый день',
#                           'рад вас видеть!'
#             ]
#         },
#         'bye': {
#             'examples': [
#                          'Пока!',
#                          'Увидимся',
#                          'Чао...'
#             ],
#             'responses': [
#                           'Покеда',
#                           'До свиданья',
#                           'Сайонара!!'
#             ]
#         }
#     },
#     'default': [
#                 'Извините, я еще глупый робот',
#                 'Извини, я ничего не понял, переформулируй, пожалуйста'
#     ]
# }

In [10]:
with open('/content/BOT_CONFIG.json', 'r') as f:
  BOT_CONFIG = json.load(f)
  
# with open('/content/BOT_CONFIG.json', 'w') as f:
#   json.dump(BOT_CONFIG, f, ensure_ascii=False, indent=3)

FileNotFoundError: ignored

In [6]:
def clean(text):
  text = text.lower()
  cleaned_text = ''
  for ch in text:
    if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя':
      cleaned_text = cleaned_text + ch
  return cleaned_text

def get_intent(text):
  for intent in BOT_CONFIG['intents'].keys():
    for example in BOT_CONFIG['intents'][intent]['examples']:
      cleaned_example = clean(example)
      cleaned_text = clean(text)
      if nltk.edit_distance(cleaned_example, cleaned_text) / max(len(cleaned_example), len(cleaned_text)) * 100 < 40:
        return intent
  return 'unknown_intent'

## Обучение модели

In [ ]:
X = []
y = []
for intent in BOT_CONFIG['intents']:
    for example in BOT_CONFIG['intents'][intent]['examples']:
        X.append(example)
        y.append(intent)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1,3)) #CountVectorizer(analyzer='char', ngram_range=(1,3), preprocessor=clean)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
clf = RidgeClassifier() #LogisticRegression()
clf.fit(X_train_vectorized, y_train)
clf.score(X_train_vectorized, y_train), clf.score(X_test_vectorized, y_test)

(0.807843137254902, 0.29965156794425085)

In [ ]:
# CountVectorizer + LogisticRegression : (0.7886710239651417, 0.2665505226480836)
# TfidfVectorizer + RidgeClassifier : (0.807843137254902, 0.29965156794425085)

In [ ]:
def get_intent_by_model(text):
  vectorized_text = vectorizer.transform([text])
  return clf.predict(vectorized_text)[0]

In [ ]:
get_intent_by_model('как дела?')

'howdoing'

## Тестирование бота

In [ ]:
def bot(text):
  intent = get_intent_by_model(text)
  
  return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
# while True:
#   input_text = input()
#   response = bot(input_text)
#   print(response)

## Подключение к Telegram

In [ ]:
# !pip install python-telegram-bot --upgrade

In [3]:
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text(bot(clean(update.message.text)))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1878285408:AAEmnm9kR55RwWlnW1suwgOragRi1CerFG8")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

ModuleNotFoundError: ignored

In [1]:
main()

NameError: ignored